Consider the current parking system of a city which charges a fixed rate for parking. The city is struggling to keep up with the increased demand. To address this issue, the city council has decided to modify the pricing scheme to better promote social welfare. In general, the city considers social welfare higher when more parking is being used, the exception being that the city prefers that at least one spot is left unoccupied (so that it is available in case someone really needs it). The city council has created a Markov decision process (MDP) to model the demand for parking with a reward function that reflects its preferences. Now the city has hired you — an expert in dynamic programming — to help determine an optimal policy.

The first cell contains that defines the environment as ParkingLot class. The transition function has already been defined that gives the probabilities of next state and reward given a certain state and action.

**Do not change the contents of the following cell.**








In [1]:

import numpy as np

class ParkingLot:
    def __init__(self, S, A):
        """
        Initialize the ParkingLot environment.
        :param S: Number of states (parking spots occupied)
        :param A: Number of actions (pricing levels)
        """
        self.S = S  # Number of states
        self.A = A  # Number of actions

    def transitions(self, state, action):
        """
        Compute the transition probabilities and rewards for a given state and action.
        :param state: The current state
        :param action: The chosen action
        :return: 2D NumPy array where each row index is next_state,
                 the first column is reward, and the second column is transition probability
        """
        # Define reward function
        reward = 5 if state == self.S - 1 else (10 if state == self.S - 2 else (-5 if state == 0 else 0))

        # Define transition probabilities based on action (pricing level)
        probabilities = np.zeros(self.S)

        # Dynamically adjust probability based on action scaling
        demand_increase_prob = max(0.7 - (0.1 * action), 0.1)  # Higher pricing reduces demand increase
        demand_decrease_prob = min(0.1 * action, 0.6)  # Higher pricing increases departure
        stay_prob = 1.0 - demand_increase_prob - demand_decrease_prob

        if state < self.S - 1:
            probabilities[state + 1] = demand_increase_prob  # Probability of increased occupancy
        if state > 0:
            probabilities[state - 1] = demand_decrease_prob  # Probability of decreased occupancy
        probabilities[state] = stay_prob  # Probability of no change

        # Create a 2D NumPy array with rewards and probabilities
        transition_matrix = np.zeros((self.S, 2))  # First column: reward, Second column: probability

        for next_state in range(self.S):
            if probabilities[next_state] > 0:
                transition_matrix[next_state] = [reward, probabilities[next_state]]

        return transition_matrix




For now, let's consider an environment with three parking spaces and three price points. Note that an environment with three parking spaces actually has four states — zero, one, two, or three spaces could be occupied. Initially the policy is equiprobable policy assignig equal probability to each action. This environment if only for showing you how to iterate over different values of policies and transition probabilities.

You can change the number of spaces and actions to see different transitions.

In [2]:
# Example usage
num_spaces = 4
num_actions = 3
env = ParkingLot(num_spaces,num_actions)

V=np.zeros(env.S)


#random policy
pi = np.full((num_spaces, num_actions), 1.0 / num_actions)  # Uniform probabilities


# for action,action_prob in enumerate(pi[2]):
#   print(f"Action: {action}: Action Probability: {action_prob}")


for state,action_probs in enumerate(pi):
  print(f"State: {state}: Action Probabilities: {action_probs}")


# Test a specific transition
state, action = 2, 1
transition_result = env.transitions(state, action)

for new_state, (reward, probability) in enumerate(transition_result):
    print(f"Next State: {new_state}, Reward: {reward}, Probability: {probability}")





State: 0: Action Probabilities: [0.33333333 0.33333333 0.33333333]
State: 1: Action Probabilities: [0.33333333 0.33333333 0.33333333]
State: 2: Action Probabilities: [0.33333333 0.33333333 0.33333333]
State: 3: Action Probabilities: [0.33333333 0.33333333 0.33333333]
Next State: 0, Reward: 0.0, Probability: 0.0
Next State: 1, Reward: 10.0, Probability: 0.1
Next State: 2, Reward: 10.0, Probability: 0.30000000000000004
Next State: 3, Reward: 10.0, Probability: 0.6


The following cells contain empty definitions of different fucntions that you need to fill in in order to find optimal policy using policy iteration and value iteration. Please make sure that the code must work for any number of states and actions. env.S gives you the number of states and env.A gives you the number of actions defined for a particular environment.

In [3]:
def evaluate_policy(V,pi,env,gamma,theta):
    while True:
        delta = 0
        for s in range(env.S):
            v = V[s]
            v_new = 0
            for a in range(env.A):
                for s_next in range(env.S):
                    transition = env.transitions(s, a)[s_next]
                    r, prob = transition[0], transition[1]
                    v_new += pi[s][a] * prob * (r + gamma * V[s_next])

            V[s] = v_new
            delta = max(delta, abs(v - V[s]))

        if delta < theta:
            break
    return V


In [4]:
def improve_policy(V,pi,env,gamma):
  policy_stable = True
  for s in range(env.S):
        old_action = np.copy(pi[s])


        q_values = np.zeros(env.A)
        for a in range(env.A):
            for s_next in range(env.S):
                transition = env.transitions(s, a)[s_next]
                r, prob = transition[0], transition[1]
                q_values[a] += prob * (r + gamma * V[s_next])


        best_action = np.argmax(q_values)


        pi[s] = np.zeros(env.A)
        pi[s][best_action] = 1.0  # Setting the probability of the best action to 1 so it gets chosen

        if not np.array_equal(pi[s], old_action):
            policy_stable = False
  return policy_stable

In [5]:
def policy_iteration(V,pi,env,gamma,theta):
    while True:
        V = evaluate_policy(V, pi, env, gamma, theta)
        policy_stable = improve_policy(V, pi, env, gamma)

        if policy_stable:
            break

    return V,pi

In [6]:
def value_iteration(V, pi, env, gamma, theta):
    while True:
        delta = 0

        best_actions = np.zeros(env.S, dtype=int)

        for s in range(env.S):
            v = V[s]

            q_values = np.zeros(env.A)
            for a in range(env.A):
                for s_next in range(env.S):
                    transition = env.transitions(s, a)[s_next]
                    r, prob = transition[0], transition[1]
                    q_values[a] += prob * (r + gamma * V[s_next])



            V[s] = np.max(q_values)
            delta = max(delta, abs(v - V[s]))
            best_actions[s] = np.argmax(q_values)
        if delta < theta:

            for s in range(env.S):
                pi[s] = np.zeros(env.A)
                pi[s][best_actions[s]] = 1.0
            break

    return V, pi

The following cell contains a new environment with different number of spaces and actions. This cell tests your implemented functions.

**Do not change the contents of the following cell.**

In [7]:
# Example usage
num_spaces = 6
num_actions = 3
env = ParkingLot(num_spaces,num_actions)
gamma = 0.9
theta = 0.1


V=np.zeros(env.S)


#random policy
pi = np.full((num_spaces, num_actions), 1.0 / num_actions)  # Uniform probabilities



V = evaluate_policy(V,pi,env,gamma,theta)

V_actual = np.array([0.98774032,9.73784006,13.03387846,16.00978726,19.47441585 ,5.14059646])

assert np.isclose(V, V_actual, atol=0.01).all(), f"Values are not within the first two decimal places: {V} vs {V_actual}"

V,pi = policy_iteration(V,pi,env,gamma,theta)
pi_actual=np.array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])
assert (pi==pi_actual).all(), f"Incorrect policy"
print("All tests passed!")







All tests passed!


The following cell contains a new environment with different number of spaces and actions. This cell tests your implemented functions.

**Do not change the contents of the following cell.**

In [8]:
# Example usage
num_spaces = 10
num_actions = 3
env = ParkingLot(num_spaces,num_actions)
gamma = 0.9
theta = 0.1


V=np.zeros(env.S)


#random policy
pi = np.full((num_spaces, num_actions), 1.0 / num_actions)  # Uniform probabilities



V = evaluate_policy(V,pi,env,gamma,theta)



V_actual = np.array([ -3.39512584,  3.85951704,  5.8585558 ,  7.31664517,  8.93320282, 10.86460577, 13.20128222, 16.03631496, 19.47875764,  5.14121156])

assert np.isclose(V, V_actual, atol=0.01).all(), f"Values are not within the first two decimal places: {V} vs {V_actual}"

V,pi = policy_iteration(V,pi,env,gamma,theta)
pi_actual=np.array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])
assert (pi==pi_actual).all(), f"Incorrect policy"
print("All tests passed!")









All tests passed!


**Bonus point**
Create a new environment in the following cell with any number of states and actions and test the implementation of your value iteration algorithm in this cell.

In [9]:

num_spaces = 9
num_actions = 3
env = ParkingLot(num_spaces, num_actions)
gamma = 0.7
theta = 0.01

V = np.zeros(env.S)
pi = np.full((num_spaces, num_actions), 1.0 / num_actions)

V, pi = value_iteration(V, pi, env, gamma, theta)

print("Optimal Value Function (V):", V)
print("Optimal Policy (pi):\n", pi)


Optimal Value Function (V): [-4.63462073  0.98029424  1.58738691  2.56271348  4.13337024  6.66477439
 10.74560493 17.32471158  6.23475288]
Optimal Policy (pi):
 [[0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]
